In [2]:
# %matplotlib notebook
import silq
silq.initialize("EWJN", mode='analysis')

from silq.tests.test_parameters import GaussianParameter, Gaussian2DParameter
from silq.measurements.measurement_modules import *
from silq.measurements.measurement_types import *

dummy_parameter = ManualParameter(name='dummy', initial_value=0)
dummy_parameter2 = ManualParameter(name='dummy2', initial_value=0)
gaussian_parameter = GaussianParameter('gaussian', set_parameter=dummy_parameter, std=4)
gaussian_2D_parameter = Gaussian2DParameter('gaussian', set_parameters=[dummy_parameter, dummy_parameter2], std=4)

qc.show_subprocess_widget()

Initializing imports
Initializing functions
Initializing data
Initializing analysis
Initializing plotting
Initialization complete


# Loop measurements

## Loop 0D measurement

In [3]:
dummy_parameter[0]

loop0D_msmt = Loop0DMeasurement('loop_0D', acquisition_parameter=gaussian_parameter,
                                discriminant='gaussian')
condition_set = ConditionSet(('gaussian', '>', 0.9))
loop0D_msmt.condition_sets = [condition_set]
data =loop0D_msmt()

loop0D_msmt.get_optimum()

DataSet:
data = qc.load_data('2017-01-04/#030_loop_0D_gaussian_11-21-08')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | single_set | single       | (1,)
   Measured | gaussian   | gaussian     | (1,)
acquired at 2017-01-04 11:21:09


({}, 1.0)

In [4]:
dummy_parameter(3)
data =loop0D_msmt()
loop0D_msmt.get_optimum()

DataSet:
data = qc.load_data('2017-01-04/#031_loop_0D_gaussian_11-21-09')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | single_set | single       | (1,)
   Measured | gaussian   | gaussian     | (1,)
acquired at 2017-01-04 11:21:09


({}, nan)

## Loop 1D measurement

In [5]:
loop1D_msmt = Loop1DMeasurement('loop_1D', acquisition_parameter=gaussian_parameter, 
                                set_parameter=dummy_parameter, discriminant='gaussian')
condition_set = ConditionSet(('gaussian', '>', 0.9))
loop1D_msmt.condition_sets = [condition_set]

data = loop1D_msmt(step=0.1, points=10)
data =loop1D_msmt()

In [6]:
data.sync()
data.gaussian

DataArray[10]: gaussian
array([ 0.666,  0.645,  0.623,  0.602,  0.58 ,  0.559,  0.538,  0.517,
        0.496,  0.475])

## Loop 2D measurement

In [7]:
loop2D_msmt = Loop2DMeasurement('loop_2D', acquisition_parameter=gaussian_2D_parameter, 
                                set_parameters=[dummy_parameter2, dummy_parameter],
                                discriminant='gaussian', silent=False)
condition_set = ConditionSet(('gaussian', '>', 0.99), update=True)
loop2D_msmt.condition_sets = [condition_set]
loop2D_msmt(step=1, points=11)
data =loop2D_msmt()

DataSet:
data = qc.load_data('2017-01-04/#033_loop_2D_dummy2_dummy_gaussian_11-21-09')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy2_set | dummy2       | (11,)
   Setpoint | dummy_set  | dummy        | (11, 11)
   Measured | gaussian   | gaussian     | (11, 11)
started at 2017-01-04 11:21:09
Updating set parameters to optimal values


In [8]:
data.sync()
qc.MatPlot(data.gaussian)

# Measurement Parameters

## Peak parameter

In [9]:
dummy_parameter2(3)


In [10]:
from silq.parameters.measurement_parameters import TrackPeakParameter

peak_parameter = TrackPeakParameter('track', 
                                    set_parameter=dummy_parameter2,
                                    acquisition_parameter=gaussian_2D_parameter,
                                    peak_width=3, points=11,
                                    discriminant='gaussian',
                                    silent=False, threshold=0.2)

In [11]:
data = qc.Loop(dummy_parameter[-3:6:0.5]).each(peak_parameter).run(name='test', background=False)

Updating set parameters to initial values
Updating set parameters to initial values
Updating set parameters to initial values
Updating set parameters to optimal values
Updating set parameters to optimal values
Updating set parameters to optimal values
Updating set parameters to optimal values
Updating set parameters to optimal values
Updating set parameters to optimal values
Updating set parameters to optimal values
Updating set parameters to optimal values
Updating set parameters to initial values
Updating set parameters to initial values
Updating set parameters to initial values
Updating set parameters to initial values
Updating set parameters to initial values
Updating set parameters to initial values
Updating set parameters to initial values
DataSet:
data = qc.load_data('2017-01-04/#034_test_11-21-10')
   mode = DataMode.LOCAL
   <Type>   | <array_id>       | <array.name>     | <array.shape>
   Setpoint | dummy_set        | dummy            | (18,)
   Measured | index0           | 

In [12]:
data.sync()
data.optimal_set_vals

DataArray[18]: optimal_set_vals
array([ nan,  nan,  nan,  1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        nan,  nan,  nan,  nan,  nan,  nan,  nan])

In [13]:
plot = qc.MatPlot(subplots=2)
plot[0].add(data.gaussian)
plot[1].add(data.optimal_set_vals)

## Calibration parameter

In [14]:
data = qc.Loop(
    dummy_parameter[-5:5:0.1]).loop(
    dummy_parameter2[-5:5:0.1]).each(
    gaussian_2D_parameter).run(
    name='test_2D', background=False)
qc.MatPlot(data.gaussian)

DataSet:
data = qc.load_data('2017-01-04/#053_test_2D_11-21-11')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy_set  | dummy        | (100,)
   Setpoint | dummy2_set | dummy2       | (100, 100)
   Measured | gaussian   | gaussian     | (100, 100)
started at 2017-01-04 11:21:13


In [ ]:
dummy_parameter(2)
dummy_parameter2(3)

In [29]:
condition_sets = [ConditionSet(('gaussian', '>', 0.4), on_success='success', update=True)]
measurements = [Loop2DMeasurement('detailed', set_parameters),
                Loop2DMeasurement('rough',set_vals=[dummy_parameter[-5:5:1], dummy_parameter2[-5:5:1]])]
measurement_sequence = MeasurementSequence('retune', measurements=measurements,
                                           acquisition_parameter=gaussian_2D_parameter,
                                           condition_sets=condition_sets, silent=False)

In [30]:
from silq.parameters.measurement_parameters import MeasurementSequenceParameter

retune_parameter = MeasurementSequenceParameter('retune_parameter', measurement_sequence,
                                                set_parameters=[dummy_parameter, dummy_parameter2])

In [31]:
retune_parameter()

Performing measurement detailed measurement
DataSet:
data = qc.load_data('2017-01-04/#058_detailed_dummy_dummy2_gaussian_11-24-37')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy_set  | dummy        | (3,)
   Setpoint | dummy2_set | dummy2       | (3, 10)
   Measured | gaussian   | gaussian     | (3, 10)
started at 2017-01-04 11:24:37
Performing measurement rough measurement
DataSet:
data = qc.load_data('2017-01-04/#059_rough_dummy_dummy2_gaussian_11-24-37')
   mode = DataMode.LOCAL
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy_set  | dummy        | (10,)
   Setpoint | dummy2_set | dummy2       | (10, 10)
   Measured | gaussian   | gaussian     | (10, 10)
started at 2017-01-04 11:24:37
Finished measurements


(2, [0, 0], 1.0)

In [33]:
dataset = measurement_sequence.datasets[0]
qc.MatPlot(dataset.gaussian)